# ML Zoomcamp 2023 - Homework #10

Name: Wong Chee Fah

Email: wongcheefah@gmail.com

## Homework

In this homework, we'll deploy the credit scoring model from the homework 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.


## Bulding the image

Clone the course repo if you haven't:

```
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
```

Go to the `course-zoomcamp/cohorts/2023/05-deployment/homework` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:hw10 .
```

> **Note:** If you have troubles building the image, you can 
> use the image we built and published to docker hub:
> `docker pull svizor/zoomcamp-model:hw10`

## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'get_credit': True, 'get_credit_probability': <value>}
```

Here `<value>` is the probability of getting a credit card. You need to choose the right one.

* 0.3269
* 0.5269
* 0.7269
* 0.9269

In [1]:
!docker build -t zoomcamp-model:hw10 .

[+] Building 0.0s (0/0)                                          docker:default
[+] Building 0.0s (0/0)                                          docker:default
[+] Building 0.0s (0/2)                                          docker:default
 => [internal] load .dockerignore                                          0.0s
 => => transferring context:                                               0.0s
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.1s (2/3)                                          docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 293B                                       0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:3.10.12-  0.0s
[+] Building 0.3s (2/3)                 

In [2]:
!docker run -it --rm -d -p 9696:9696 zoomcamp-model:hw10

bfdc0ed8dde4a6315f6ebe67575f338a70b84defe1f827a2c2cd6618ee968a00


In [3]:
!python3 q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}


In [4]:
!docker ps

CONTAINER ID   IMAGE                 COMMAND                  CREATED          STATUS          PORTS                    NAMES
bfdc0ed8dde4   zoomcamp-model:hw10   "waitress-serve --li…"   32 seconds ago   Up 30 seconds   0.0.0.0:9696->9696/tcp   unruffled_feistel


In [5]:
!docker rm -f unruffled_feistel

unruffled_feistel


In [6]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


##### Ans: `0.7269`

## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

## Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.

In [7]:
!kind --version

kind version 0.20.0


##### Ans: `0.20.0`

## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```

In [8]:
!kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.27.3) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a nice day! 👋


In [9]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:36641
CoreDNS is running at https://127.0.0.1:36641/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

Now let's test if everything works. Use `kubectl` to get the list of running services. 

What's `CLUSTER-IP` of the service that is already running there? 

In [10]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   60m


##### Ans: `10.96.0.1`

## Question 4

To be able to use the docker image we previously created (`zoomcamp-model:hw10`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`

In [11]:
!kind load docker-image zoomcamp-model:hw10

Image: "zoomcamp-model:hw10" with ID "sha256:7192d843e24e90fa8570fb91e3e11c0474a28c93feccb5cf5655b856095c8a53" not yet present on node "kind-control-plane", loading...


In [12]:
!kind get nodes

kind-control-plane


##### Ans: `kind load docker-image`

## Question 5

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit
spec:
  selector:
    matchLabels:
      app: credit
  replicas: 1
  template:
    metadata:
      labels:
        app: credit
    spec:
      containers:
      - name: credit
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

In [13]:
!kubectl apply -f deployment.yaml

deployment.apps/credit created


In [14]:
!kubectl get deployment

NAME     READY   UP-TO-DATE   AVAILABLE   AGE
credit   1/1     1            1           12s


In [15]:
!kubectl get pod

NAME                      READY   STATUS    RESTARTS   AGE
credit-65ccb78b69-wjb8c   1/1     Running   0          17s


##### Ans: `9696`

## Question 6

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file.

In [16]:
!kubectl apply -f service.yaml

service/credit created


In [17]:
!kubectl get service

NAME         TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
credit       LoadBalancer   10.96.43.185   <pending>     80:30904/TCP   13s
kubernetes   ClusterIP      10.96.0.1      <none>        443/TCP        70m


##### Ans: `credit`

## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

In [18]:
!kubectl port-forward service/credit 9696:80

Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696
^C


In [19]:
# Using subprocess module to run the port-forwarding command in the background
import subprocess

command = [
    "kubectl",
    "port-forward",
    "service/credit",
    "9696:80"
]

with open("forward.log", "w") as log_file:
    subprocess.Popen(command, stdout=log_file, stderr=log_file)

In [20]:
!python3 q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}


## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment credit --name credit-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-hpa   Deployment/credit   1%/20%    1         3         1          27s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0% as there are no clients sending requests to the server.
> 
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release 
> from the `components.yaml` manifest:
> ```bash
> kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```

In [21]:
!kubectl autoscale deployment credit --name credit-hpa --cpu-percent=20 --min=1 --max=3

horizontalpodautoscaler.autoscaling/credit-hpa autoscaled


In [26]:
!kubectl get hpa

NAME         REFERENCE           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-hpa   Deployment/credit   <unknown>/20%   1         3         1          25s


## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing
`q6_test.py` script by putting the operator that sends the request to the credit service into a loop.

```python
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.

In [27]:
!python3 q6_test_loop.py > /dev/null

^C
Traceback (most recent call last):
  File "/home/wongcheefah/learning/zoomcamp/machine-learning-zoomcamp/ml-zoomcamp/10-kubernetes/homework/q6_test_loop.py", line 11, in <module>
    response = requests.post(url, json=client).json()
  File "/home/wongcheefah/anaconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/requests/api.py", line 115, in post
    return request("post", url, data=data, json=json, **kwargs)
  File "/home/wongcheefah/anaconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/requests/api.py", line 59, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/wongcheefah/anaconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/requests/sessions.py", line 589, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/wongcheefah/anaconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/requests/sessions.py", line 703, in send
    r = adapter.send(request, **kwargs)
  File "/home/wongcheefah/anaconda3/envs/ml-zoomcamp/lib/python3.9/site-pa

In [27]:
command = [
    "python3",
    "q6_test_loop.py"
]

with open("q6_test_loop.log", "w") as log_file:
    subprocess.Popen(command, stdout=log_file, stderr=log_file)

## Question 7 (optional)

Run `kubectl get hpa credit-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* 1
* 2
* 3
* 4

> Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled 
> in any way it may happen that the final number of replicas will differ from initial.

In [31]:
!kubectl apply -f components.yaml

serviceaccount/metrics-server unchanged
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader unchanged
clusterrole.rbac.authorization.k8s.io/system:metrics-server unchanged
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader unchanged
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator unchanged
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server unchanged
service/metrics-server unchanged
deployment.apps/metrics-server configured
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io unchanged


In [32]:
!kubectl get hpa credit-hpa --watch

NAME         REFERENCE           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-hpa   Deployment/credit   <unknown>/20%   1         3         1          27m
credit-hpa   Deployment/credit   23%/20%         1         3         1          28m
credit-hpa   Deployment/credit   21%/20%         1         3         2          28m
credit-hpa   Deployment/credit   24%/20%         1         3         2          28m
credit-hpa   Deployment/credit   30%/20%         1         3         2          29m
credit-hpa   Deployment/credit   12%/20%         1         3         3          29m
credit-hpa   Deployment/credit   14%/20%         1         3         3          29m
credit-hpa   Deployment/credit   8%/20%          1         3         3          30m
credit-hpa   Deployment/credit   9%/20%          1         3         3          31m
credit-hpa   Deployment/credit   7%/20%          1         3         3          31m
credit-hpa   Deployment/credit   8%/20%          1         3         3      

In [34]:
!ps aux | grep "python3 q6_test_loop.py"

wongche+   49334  3.3  0.2  27692 22640 pts/5    S+   22:13   1:10 python3 q6_test_loop.py
wongche+   92734 72.0  0.0   4784  3224 pts/10   Ss+  22:48   0:00 /bin/bash -c ps aux | grep "python3 q6_test_loop.py"
wongche+   92743  0.0  0.0   4040  2080 pts/10   S+   22:48   0:00 grep python3 q6_test_loop.py


In [35]:
!kill 49334

##### Ans: `3`